<a href="https://colab.research.google.com/github/reknahs/email-anonymization/blob/main/azure_anonymization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.7/328.7 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.4 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [16]:
%%capture
from google.colab import userdata
API_VERSION = userdata.get('API_VERSION')
API_KEY = userdata.get('API_KEY')
AZURE_ENDPOINT = userdata.get('AZURE_ENDPOINT')
DEPLOYMENT_NAME = userdata.get('DEPLOYMENT_NAME')

In [4]:
# Path to .eml files

# 5 cancellations
cancellations = ["/content/drive/MyDrive/Extraction examples/Cancellations/cancellations"+str(i)+".eml" for i in range(1,6)]

# 5 newsletters
newsletters = ["/content/drive/MyDrive/Extraction examples/Newsletters/newsletters"+str(i)+".eml" for i in range(1,6)]

# 5 offers, coupons, and sales
ocs = ["/content/drive/MyDrive/Extraction examples/Offers, Coupons, Sales/ocs"+str(i)+".eml" for i in range(1,6)]

# 1 return
refunds = ["/content/drive/MyDrive/Extraction examples/Refunds/refunds1.eml"]

# 3 orders
orders = ["/content/drive/MyDrive/Extraction examples/Orders/orders"+str(i)+".eml" for i in range(1,4)]

In [5]:
import re

def remove_links(text):
    # Regular expression pattern to match URLs
    url_pattern = re.compile(r'https?://\S+|www\.\S+')

    # Replace all matches with an empty string
    text_without_links = re.sub(url_pattern, '', text)

    return text_without_links

In [6]:
import email
from email import policy
from email.parser import BytesParser

def get_email_body(email_path):
  # Read the .eml file
  with open(email_path, 'rb') as f:
      msg = BytesParser(policy=policy.default).parse(f)

  # Extract subject
  subject = msg['subject']
  print('Subject:', subject)

  # Extract body (both plain text and HTML)
  body = ""
  if msg.is_multipart():
      for part in msg.iter_parts():
          content_type = part.get_content_type()
          content_disposition = str(part.get('Content-Disposition'))

          # Look for plain text parts
          if content_type == 'text/plain' and 'attachment' not in content_disposition:
              body = part.get_payload(decode=True).decode(part.get_content_charset())
              break
          # Look for HTML parts
          elif content_type == 'text/html' and 'attachment' not in content_disposition:
              body = part.get_payload(decode=True).decode(part.get_content_charset())
              break
  else:
      # If the message isn't multipart, the payload is simple text
      body = msg.get_payload(decode=True).decode(msg.get_content_charset())
  return subject+"\n\n"+body

for i in range(len(cancellations)):
  cancellations[i] = remove_links(get_email_body(cancellations[i]))
for i in range(len(newsletters)):
  newsletters[i] = remove_links(get_email_body(newsletters[i]))
for i in range(len(ocs)):
  ocs[i] = remove_links(get_email_body(ocs[i]))
for i in range(len(refunds)):
  refunds[i] = remove_links(get_email_body(refunds[i]))
for i in range(len(orders)):
  orders[i] = remove_links(get_email_body(orders[i]))

Subject: Order #1672 has been canceled
Subject: Order #1673 has been canceled
Subject: Order #1754 has been canceled
Subject: Order #1736 has been canceled
Subject: Order #1732 has been canceled
Subject: Dwight Yoakam, Jo Koy, Monster Jam & more near you!
Subject: Exercises for leg, hip and core stability
Subject: Red Dot Your Fabric Now...
Subject: Two Tree Styles, One Summer Vibe 🌳🌳
Subject: This World Cup, win big with Crickit!🎁
Subject: Buy One, Get One FREE ♨️ Baby Back Ribs
Subject: Don't Miss Out! 20% Off Lasers - Shop Now!
Subject: It’s Giving Ribbed
Subject: Just Dropped! Your Summer Capsule Wardrobe
Subject: ✨50% Off✨ My Body My Choice Unisex Tee
Subject: Subject: Flip It and Reverse It (Refund On The Way)
Subject: Your Amazon.in order #407-1283161-2007545 of 1 item has been dispatched
Subject: Fwd: FW: Order #1691 confirmed
Subject: Fwd: FW: Order #1687 confirmed


In [17]:
import os
from openai import AzureOpenAI

client = AzureOpenAI(
    api_key=API_KEY,
    api_version=API_VERSION,
    azure_endpoint = AZURE_ENDPOINT
    )

deployment_name= DEPLOYMENT_NAME

# Send a completion call to generate an answer
prompt = cancellations[0]
content = '''
Your job is to output a given email but after swapping ONLY PII with fake info. PII is anything that can give away someone's identity.

CHANGE THE ORDER NUMBER NO MATTER WHAT.

For example, you NEED to REPLACE ALL instances of the following for the CUSTOMER ONLY:
-Name
-Billing/shipping address
-Phone number
-Email address
-Order number
-Any portion of credit card number

Replace the above with fake names/addresses/phone numbers/emails/order numbers/credit card numbers/etc. Do not just delete without replacing. Do not change dollar amounts or product names.
Example:

Input:
Hi James,

We'd like to confirm your order (Order number: 6823) to the 8523 Main St. San Francisco, CA 92356.
Paid with Mastercard ending in 9823.

Example of your output:
Hi Stephen,

We'd like to confirm your order (Order number: 9866) to the 1323 Oak St. Orlando, FL 12235.
Paid with Mastercard ending in 0912.
'''
response = client.chat.completions.create(
    model=deployment_name,
    messages=[{"role": "system", "content": content},
              {"role": "user", "content": prompt}
              ],
    temperature=0
)
print(response.choices[0].message)

ChatCompletionMessage(content='Order #9865 has been canceled\n\nYour order has been canceled \n \n \n \n < \t \n Order #9865 \n \n \n\nYour order has been canceled \n\n Order #9865 was canceled at your request and your payment has not yet been refunded. \n\n \n \n \n\nRemoved Items \n \n \n < \t Velvet Elegance: Premium Merlot Wine\xa0×\xa01 Refunded \t \n\n $1.00 \n\n \n \t \n Subtotal \n\n \t $1.00 \n Shipping \n\n \t $4.90 \n Taxes \n\n \t $0.00 \n Total \n\n \t $5.90 USD \n \n If you have any questions, reply to this email or contact us at orders@curativelifestyle.com <mailto:orders@curativelifestyle.com> \n\n <', role='assistant', function_call=None, tool_calls=None)


In [18]:
print(response.choices[0].message.content)

Order #9865 has been canceled

Your order has been canceled 
 
 
 
 < 	 
 Order #9865 
 
 

Your order has been canceled 

 Order #9865 was canceled at your request and your payment has not yet been refunded. 

 
 
 

Removed Items 
 
 
 < 	 Velvet Elegance: Premium Merlot Wine × 1 Refunded 	 

 $1.00 

 
 	 
 Subtotal 

 	 $1.00 
 Shipping 

 	 $4.90 
 Taxes 

 	 $0.00 
 Total 

 	 $5.90 USD 
 
 If you have any questions, reply to this email or contact us at orders@curativelifestyle.com <mailto:orders@curativelifestyle.com> 

 <


In [19]:
print(cancellations[0])

Order #1672 has been canceled

Your order has been canceled 
 
 
 
 < 	 
 Order #1672 
 
 

Your order has been canceled 

 Order #1672 was canceled at your request and your payment has not yet been refunded. 

 
 
 

Removed Items 
 
 
 < 	 Velvet Elegance: Premium Merlot Wine × 1 Refunded 	 

 $1.00 

 
 	 
 Subtotal 

 	 $1.00 
 Shipping 

 	 $4.90 
 Taxes 

 	 $0.00 
 Total 

 	 $5.90 USD 
 
 If you have any questions, reply to this email or contact us at orders@curativelifestyle.com <mailto:orders@curativelifestyle.com> 

 < 


In [20]:
# 3. tool_calls
def chat_completion_request(messages, tools=None, tool_choice=None, model=deployment_name):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice=tool_choice,
            temperature=1
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

In [21]:
schema = [
  {
    "type": "function",
    "function": {
      "name": "orders_details_email_header",
      "description": "Send important details of an email to a client",
      "parameters": {
        "type": "object",
        "properties": {
          "SenderDomain": {
            "type": "string",
            "description": "The domain name (without the subdomain) of the sender of the email. If the email was sent on the behalf of a third party, this should be the domain name of the third party."
          },
          "OrderId": {
            "type": "string",
            "description": "The order id/order number of the order the email contains"
          },
          "OrderDate": {
            "type": "string",
            "description": "The date that the order the email contains was sent, in MM/DD/YYYY format"
          },
          "OrderCurrency": {
            "type": "string",
            "description": "The currency used in the order the email contains, in ISO 4217 3-letter currency code format."
          },
          "OrderLineItems": {
            "type": "string",
            "description": "A list containing any and all line items contained in the order from the email. Each line item should follow the format of 'Item Name: Item Quantity x Unit Price'. The unit price be a number only, and not include a currency code."
          },
          "OrderTax": {
            "type": "string",
            "description": "The tax paid for the order the email contains. This value should be a number only, and not include a currency code or symbol."
          },
          "OrderTotal": {
            "type": "string",
            "description": "The total amount paid for the order the email contains. This value should be a number only, and not include a currency code or symbol"
          },
          "PaymentMethod": {
            "type": "string",
            "description": "The general method or scheme used for performing the payment associated with the order. Examples are 'Visa', 'MasterCard', 'PayPal' etc."
          },
          "PaymentInstrument": {
            "type": "string",
            "description": "The specific instance of a Payment Method used for the order. Examples are 'Visa card ending in 1234', 'MasterCard ending in 2345', 'PayPal account with username john.doe@example.com'"
          },
          "ShippingMethod": {
            "type": "string",
            "description": "The shipping method used for the order"
          },
          "ExpectedDeliveryDate": {
            "type": "string",
            "description": "The expected delivery date of the order in MM/DD/YYYY format"
          },
          "SchemaVersion": {
            "type": "string",
            "description": "This value should always be 1.0"
          }
        },
        "required": [
          "SchemaVersion"
        ]
      }
    }
  },
  {
    "type": "function",
    "function": {
      "name": "offers_coupons_details_email_header",
      "description": "Send important details of an email to a client",
      "parameters": {
        "type": "object",
        "properties": {
          "SenderDomain": {
            "type": "string",
            "description": "The domain name (without the subdomain) of the sender of the email. If the email was sent on the behalf of a third party, this should be the domain name of the third party."
          },
          "OfferDescription": {
            "type": "string",
            "description": "The description of the offer contained in the email, containing the item(s) the offer is for and the coupon code if present"
          },
          "ExpirationDate": {
            "type": "string",
            "description": "The expiration date of the offer contained in the email, in MM/DD/YYYY format"
          },
          "SchemaVersion": {
            "type": "string",
            "description": "This value should always be 1.0"
          }
        },
        "required": [
          "SchemaVersion"
        ]
      }
    }
  },
  {
    "type": "function",
    "function": {
      "name": "refunds_details_email_header",
      "description": "Send important details of an email to a client",
      "parameters": {
        "type": "object",
        "properties": {
          "SenderDomain": {
            "type": "string",
            "description": "The domain name (without the subdomain) of the sender of the email. If the email was sent on the behalf of a third party, this should be the domain name of the third party."
          },
          "OrderId": {
            "type": "string",
            "description": "The order id/order number of the order the email contains a refund for"
          },
          "RefundDate": {
            "type": "string",
            "description": "The date that the refund contained in the email was send, in MM/DD/YYYY format"
          },
          "RefundCurrency": {
            "type": "string",
            "description": "The currency used in the refund the email contains, in ISO 4217 3-letter currency code format."
          },
          "RefundTotal": {
            "type": "string",
            "description": "The total amount refunded from the refund the email contains. This value should be a number only, and not include a currency code or symbol"
          },
          "RefundPaymentInstrument": {
            "type": "string",
            "description": "The specific instance of a Payment Method the refund was sent to. Examples are 'Visa card ending in 1234', 'MasterCard ending in 2345', 'PayPal account with username john.doe@example.com'"
          },
          "SchemaVersion": {
            "type": "string",
            "description": "This value should always be 1.0"
          }
        },
        "required": [
          "SchemaVersion"
        ]
      }
    }
  },
  {
    "type": "function",
    "function": {
      "name": "cancellations_details_email_header",
      "description": "Send important details of an email to a client",
      "parameters": {
        "type": "object",
        "properties": {
          "SenderDomain": {
            "type": "string",
            "description": "The domain name (without the subdomain) of the sender of the email. If the email was sent on the behalf of a third party, this should be the domain name of the third party."
          },
          "OrderId": {
            "type": "string",
            "description": "The order id/order number of the order the email contains a cancellation for"
          },
          "CancellationDate": {
            "type": "string",
            "description": "The date that the cancellation contained in the email was send, in MM/DD/YYYY format"
          },
          "RefundCurrency": {
            "type": "string",
            "description": "The currency used in the refund the email contains, in ISO 4217 3-letter currency code format."
          },
          "RefundTotal": {
            "type": "string",
            "description": "The total amount refunded from the refund the email contains. This value should be a number only, and not include a currency code or symbol"
          },
          "RefundPaymentInstrument": {
            "type": "string",
            "description": "The specific instance of a Payment Method the refund was sent to, if present. Examples are 'Visa card ending in 1234', 'MasterCard ending in 2345', 'PayPal account with username john.doe@example.com'"
          },
          "RefundDate": {
            "type": "string",
            "description": "The date that the refund contained in the email is estimated to be send if present, in MM/DD/YYYY format"
          },
          "SchemaVersion": {
            "type": "string",
            "description": "This value should always be 1.0"
          }
        },
        "required": [
          "SchemaVersion"
        ]
      }
    }
  }
]

In [22]:
messages = []
user = '''
First classify the given email into the following (it might fit multiple categories) If stuck between two categoreies, choose both:
1. Order
2. Offer, Coupon, or Sale
3. Refund
4. Cancellation
5. Receipt or Invoice
6. Shipping and Tracking Email
7. Newsletter

Remember this during classfication: newsletters are directed towards establishing a personal relationship rather than making sales.

Output the category(s) of the email.
If ALL of the category(s) of the email are in options 5-7, stop and do nothing else.
If the email has multiple categories in options 1-4, let its category be the one higher up in the list. For example, if it was both a refund and cancellation, let it be a refund (3 < 4).
Now send back a json file of the important details of that email based on the category.

Here is the email:
'''
messages.append({"role": "system", "content": "You are an AI assistant that helps users extract information from emails. Use the provided tools to extract relevant details."})
messages.append({"role": "user", "content": user+" "+orders[2]})
chat_response = chat_completion_request(
    messages, tools=schema, tool_choice={"type": "function", "function": {"name": "orders_details_email_header"}}
)
chat_response.choices[0].message

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_aKg4kcDnL9bhXGBnFPyfZnoV', function=Function(arguments='{\n    SenderDomain: "Node Next Gen Store",\n    OrderId: "1687",\n    OrderDate: null,  // Date not provided in email\n    OrderCurrency: "USD",\n    OrderLineItems: "Selling Plans Ski Wax: 1 x 24.95",\n    OrderTax: "2.34",\n    OrderTotal: "27.65",\n    PaymentMethod: null,  // Payment method not provided in email\n    PaymentInstrument: "ending with 1111",\n    ShippingMethod: "International Shipping",\n    ExpectedDeliveryDate: null,  // Delivery date not provided in email\n    SchemaVersion: "1.0"\n}', name='orders_details_email_header'), type='function')])

In [23]:
print(chat_response.choices[0].message.tool_calls)

[ChatCompletionMessageToolCall(id='call_aKg4kcDnL9bhXGBnFPyfZnoV', function=Function(arguments='{\n    SenderDomain: "Node Next Gen Store",\n    OrderId: "1687",\n    OrderDate: null,  // Date not provided in email\n    OrderCurrency: "USD",\n    OrderLineItems: "Selling Plans Ski Wax: 1 x 24.95",\n    OrderTax: "2.34",\n    OrderTotal: "27.65",\n    PaymentMethod: null,  // Payment method not provided in email\n    PaymentInstrument: "ending with 1111",\n    ShippingMethod: "International Shipping",\n    ExpectedDeliveryDate: null,  // Delivery date not provided in email\n    SchemaVersion: "1.0"\n}', name='orders_details_email_header'), type='function')]


In [10]:
f = '''/content/drive/MyDrive/mailbox/#SHEIN 11.11 Sale! 💸 FREE SHIPPING ⏳ LAST FEW HOURS .eml'''


print(remove_links(get_email_body(f)))

Subject: #SHEIN 11.11 Sale! 💸 FREE SHIPPING ⏳ LAST FEW HOURS 
#SHEIN 11.11 Sale! 💸 FREE SHIPPING ⏳ LAST FEW HOURS 

<!DOCTYPE html>
<html lang="en" xmlns:v="urn:schemas-microsoft-com:vml"
xmlns:o="urn:schemas-microsoft-com:office:office">
<head>


<meta charset="utf-8">
<meta http-equiv="X-UA-Compatible" content="IE=Edge, chrome=1">
<meta name="viewport" content="width=device-width, initial-scale=1.0,
maximum-scale=1.0, user-scalable=no">
<meta name="format-detection" content="telephone=no">
<meta name="format-detection" content="date=no">
<meta name="format-detection" content="address=no">
<meta name="format-detection" content="email=no">
<meta name="x-apple-disable-message-reformatting">



<title>SHEIN</title>
<!--[if mso]>
<xml>
<o:OfficeDocumentSettings>
<o:AllowPNG/>
<o:PixelsPerInch>96</o:PixelsPerInch>
</o:OfficeDocumentSettings>
</xml>
<![endif]-->
<style>  

/* CLIENT-SPECIFIC STYLES */
	

/* ANDROID CENTER FIX */
div[style*="margin: 16px 0;"] {
margin: 0 !important;
}
body, 

In [15]:
f = '''/content/drive/MyDrive/mailbox/"SEIT 1898" – RIMOWA's 125th Anniversary Exhibition.eml'''


print(remove_links(get_email_body(f)))

Subject: "SEIT 1898" – RIMOWA's 125th Anniversary Exhibition
"SEIT 1898" – RIMOWA's 125th Anniversary Exhibition


Pre-register for priority entry
  &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj;
 &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zwnj; &zw

In [13]:
f = '''/content/drive/MyDrive/mailbox/Our Best Deal EVER?.eml'''


print(remove_links(get_email_body(f)))

Subject: Our Best Deal EVER?
Our Best Deal EVER?

Make 2024 unforgettable. Start with this exclusive offer!

*************************
New Year’s Challenge Sale
*************************

Get 2 Years Of Stock Advisor for 80% Off*

Commit To Your Goals With 80% Off*! 
(  )


*************************
New Year’s Challenge Sale
*************************

Get 2 Years Of Stock Advisor for 80% Off*

Commit To Your Goals With 80% Off*! 
(  )
         This exclusive sale is your opportunity to commit to
any financial goals you have in 2024, by saving on the same
service that has consistently beat the S&P 500 by over 3.5X.

Click here to unlock your pathway to financial success 
(  )
 with Stock Advisor at an unprecedented 80% off* – including 300
of our historical picks and our 10 timely buys right now.

When you join our New Year's Challenge today, enjoy these 7 Free
Bonus Reports!

Bonus 1: CITIES OF THE FUTURE

Discover the six companies we feel are poised for the smart-city
boom.

Bonus 2:

In [12]:
f = '''/content/drive/MyDrive/mailbox/Order #2313 confirmed.eml'''

print(remove_links(get_email_body(f)))

Subject: Order #2313 confirmed
Order #2313 confirmed

Thank you for your purchase!

*****************************************************************
Node Next Gen Test 
(  )

*****************************************************************

Order #2313

----------------------------
Thank you for your purchase!
----------------------------

We're getting your order ready to be shipped. We will
notify you when it has been sent.

View your order 
(  )


or Visit our store 
(  )


Order summary
-------------

The Complete Snowboard × 1

Ice

Rs. 59,100.00

Subtotal

Rs. 59,100.00

Shipping

Rs. 0.00

Taxes

Rs. 10,637.61

Total

Rs. 69,737.61

Customer information
--------------------

Shipping address
----------------

Manual Entry
Super OYO Townhouse 600 Gangothri Rashtrapati Road Maruthi Veedhi
Shivaji Nagar
500090 Hyderabad TS
India

Billing address
---------------

Manual Entry
Super OYO Townhouse 600 Gangothri Rashtrapati Road Maruthi Veedhi
Shivaji Nagar
500090 Hyderabad TS
India
